In [7]:
!pip install protobuf==3.20.3

  Using cached protobuf-3.20.3-cp39-cp39-win_amd64.whl (904 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.10.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
tensorboard 2.10.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
mysql-connector-python 8.1.0 requires protobuf<=4.21.12,>=4.21.1, but you have protobuf 3.20.3 which is incompatible.
grpcio-status 1.66.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.


In [19]:
# simulate_predictors.py
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from collections import deque, defaultdict
import os

# --- Basic Predictor Implementations (Keep 1-Bit, 2-Bit, 2-Level, Tournament as before) ---
class OneBitPredictor:
    def __init__(self):
        self.states = defaultdict(lambda: 0)
    def predict(self, pc): return self.states[pc]
    def update(self, pc, actual_outcome):
        if self.predict(pc) != actual_outcome: self.states[pc] = 1 - self.states[pc]

class TwoBitPredictor:
    def __init__(self):
        self.counters = defaultdict(lambda: 0)
    def predict(self, pc): return 1 if self.counters[pc] >= 2 else 0
    def update(self, pc, actual_outcome):
        state = self.counters[pc]
        if actual_outcome == 1: self.counters[pc] = min(state + 1, 3)
        else: self.counters[pc] = max(state - 1, 0)

class TwoLevelPredictor:
    def __init__(self, ghr_bits=4, pht_size=16):
        self.ghr_bits = ghr_bits
        self.ghr = deque([0] * ghr_bits, maxlen=ghr_bits)
        self.pht = defaultdict(lambda: 0)
        self.pht_size = pht_size
    def _get_pht_index(self, pc):
        ghr_val = 0
        for bit in self.ghr: ghr_val = (ghr_val << 1) | bit
        pc_masked = pc & (self.pht_size - 1)
        return pc_masked ^ ghr_val
    def predict(self, pc): return 1 if self.pht[self._get_pht_index(pc)] >= 2 else 0
    def update(self, pc, actual_outcome):
        index = self._get_pht_index(pc)
        state = self.pht[index]
        if actual_outcome == 1: self.pht[index] = min(state + 1, 3)
        else: self.pht[index] = max(state - 1, 0)
        self.ghr.appendleft(actual_outcome)

class TournamentPredictor:
    def __init__(self, ghr_bits=4, pht_size=16, num_selectors=16):
        self.local_predictor = TwoBitPredictor()
        self.global_predictor = TwoLevelPredictor(ghr_bits, pht_size)
        self.selectors = defaultdict(lambda: 0)
        self.num_selectors = num_selectors
    def _get_selector_index(self, pc): return pc & (self.num_selectors - 1)
    def predict(self, pc):
        selector_state = self.selectors[self._get_selector_index(pc)]
        return self.global_predictor.predict(pc) if selector_state >= 2 else self.local_predictor.predict(pc)
    def update(self, pc, actual_outcome):
        local_pred = self.local_predictor.predict(pc)
        global_pred = self.global_predictor.predict(pc)
        self.local_predictor.update(pc, actual_outcome)
        self.global_predictor.update(pc, actual_outcome)
        local_correct = (local_pred == actual_outcome)
        global_correct = (global_pred == actual_outcome)
        selector_index = self._get_selector_index(pc)
        selector_state = self.selectors[selector_index]
        if local_correct and not global_correct: self.selectors[selector_index] = max(selector_state - 1, 0)
        elif not local_correct and global_correct: self.selectors[selector_index] = min(selector_state + 1, 3)

# --- Modified Fusion Predictor (Handles one ML model as predictor2) ---
class FusionPredictorMLAware:
    """Combines predictions from one basic (predictor1) and one ML (predictor2) predictor."""
    def __init__(self, predictor1_instance, predictor2_instance, ml_model_name="ML Model", num_selectors=16):
        # Store INSTANCES of the base predictors
        self.predictor1 = predictor1_instance
        self.predictor2 = predictor2_instance # This is the Keras model instance
        self.p1_name = predictor1_instance.__class__.__name__
        self.p2_name = ml_model_name # Use provided name for ML

        # Selector chooses predictor1 (0,1) or predictor2 (2,3)
        self.selectors = defaultdict(lambda: 0) # Default to predictor1
        self.num_selectors = num_selectors

    def _get_selector_index(self, pc):
         return pc & (self.num_selectors - 1)

    # PREDICT method now needs the history buffer for the ML model
    def predict(self, pc, history_buffer, history_window_size):
        selector_index = self._get_selector_index(pc)
        selector_state = self.selectors[selector_index]

        # --- Get Prediction 1 (Basic) ---
        pred1 = self.predictor1.predict(pc)

        # --- Get Prediction 2 (ML) ---
        pred2 = 0 # Default prediction if ML can't run yet
        can_predict_ml = (history_buffer is not None and len(history_buffer) >= history_window_size)
        if can_predict_ml:
            current_window = np.array(history_buffer).reshape(1, history_window_size, 1)
            # Use the stored ML model instance directly
            prob_taken = self.predictor2.predict(current_window, verbose=0)[0][0]
            pred2 = 1 if prob_taken >= 0.5 else 0

        # --- Choose based on selector ---
        if selector_state < 2: # Prefer Predictor 1
            return pred1
        else: # Prefer Predictor 2
            # If ML couldn't predict, fall back to predictor 1's prediction maybe?
            # Or just return the default pred2 (0). Let's return pred2.
            return pred2

    # UPDATE method also needs history buffer to determine ML prediction correctness
    def update(self, pc, actual_outcome, history_buffer, history_window_size):
        # --- Re-calculate predictions based on state *before* update ---
        # Get P1 prediction
        pred1 = self.predictor1.predict(pc)

        # Get P2 (ML) prediction for correctness check
        pred2 = 0 # Default if cannot predict
        can_predict_ml = (history_buffer is not None and len(history_buffer) >= history_window_size)
        if can_predict_ml:
             current_window = np.array(history_buffer).reshape(1, history_window_size, 1)
             prob_taken = self.predictor2.predict(current_window, verbose=0)[0][0]
             pred2 = 1 if prob_taken >= 0.5 else 0

        # --- Update the basic predictor's internal state ---
        self.predictor1.update(pc, actual_outcome)
        # (ML predictor has no state update here)

        # --- Update the selector ---
        # Check correctness based on the predictions made *before* the update
        p1_correct = (pred1 == actual_outcome)
        # Only consider p2 correct if it could actually make a prediction
        p2_correct = (pred2 == actual_outcome) and can_predict_ml

        selector_index = self._get_selector_index(pc)
        selector_state = self.selectors[selector_index]

        # Adjust selector state
        if p1_correct and not p2_correct:
            # P1 right, P2 wrong/unable -> Decrement (prefer P1)
            self.selectors[selector_index] = max(selector_state - 1, 0)
        elif not p1_correct and p2_correct:
            # P2 right, P1 wrong -> Increment (prefer P2)
            self.selectors[selector_index] = min(selector_state + 1, 3)
        # If both right, both wrong, or P2 unable & P1 wrong, selector state remains unchanged
# --- Simplified BranchNet Implementations (Keras/TF) ---

def build_simple_branchnet_cnn(history_window_size, num_features=1):
    """Builds a purely CNN-based model, slightly enhanced."""
    model = keras.Sequential(
        [
            keras.Input(shape=(history_window_size, num_features)),
            layers.Conv1D(filters=32, kernel_size=5, padding='causal', activation="relu"),
            layers.BatchNormalization(),
            layers.Conv1D(filters=64, kernel_size=5, padding='causal', activation="relu"),
            layers.BatchNormalization(),
            layers.MaxPooling1D(pool_size=2), # Added Pooling
            layers.Conv1D(filters=64, kernel_size=3, padding='causal', activation="relu"), # Added Layer
            layers.GlobalAveragePooling1D(),
            layers.Dense(32, activation="relu"),
            layers.Dropout(0.3),
            layers.Dense(1, activation="sigmoid"),
        ]
    )
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001) # Maybe default LR is fine here
    model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model

def build_simple_branchnet_lstm(history_window_size, num_features=1):
    """Builds a CNN+LSTM model."""
    model = keras.Sequential(
        [
            keras.Input(shape=(history_window_size, num_features)),
            layers.Conv1D(filters=32, kernel_size=5, padding='causal', activation="relu"),
            layers.BatchNormalization(),
            layers.Conv1D(filters=64, kernel_size=5, padding='causal', activation="relu"),
            layers.BatchNormalization(),
            # Feed sequence output of CNNs to LSTM
            layers.LSTM(32, return_sequences=False), # return_sequences=False gives only the last output
            layers.Dense(32, activation="relu"),
            layers.Dropout(0.3),
            layers.Dense(1, activation="sigmoid"),
        ]
    )
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005) # Often lower LR for RNNs
    model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model

def build_simple_branchnet_transformer(history_window_size, num_features=1, head_size=64, num_heads=4, ff_dim=32, num_transformer_blocks=1):
    """Builds a simple Transformer Encoder based model."""
    inputs = keras.Input(shape=(history_window_size, num_features))
    x = inputs
    for _ in range(num_transformer_blocks):
        # Attention and Normalization
        attention_output = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=0.1)(x, x)
        x = layers.Add()([x, attention_output])
        x = layers.LayerNormalization(epsilon=1e-6)(x)
        # Feed Forward Part
        ffn_output = layers.Dense(ff_dim, activation="relu")(x)
        ffn_output = layers.Dense(num_features)(ffn_output) # Project back to input dims
        x = layers.Add()([x, ffn_output])
        x = layers.LayerNormalization(epsilon=1e-6)(x)

    # Pooling or Flattening before final classification
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(16, activation="relu")(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)

    model = keras.Model(inputs=inputs, outputs=outputs)
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)
    model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model


# --- Simulation Function ---

# --- Simulation Function (Modified for ML-aware Fusion) ---

def simulate_predictors(full_history, basic_predictors, ml_models, fusion_predictor=None, history_window_size=16):
    
    """Simulates predictors and returns their accuracy. Assumes fusion_predictor uses ML."""
    # Initialize results dictionaries
    results = {name: {'correct': 0, 'total': 0} for name in basic_predictors}
    results.update({name: {'correct': 0, 'total': 0} for name in ml_models})
    
    
    if fusion_predictor:
        # Use the ML model name provided during fusion predictor init
        fusion_label = f"Fusion ({fusion_predictor.p1_name}+{fusion_predictor.p2_name})"
        results[fusion_label] = {'correct': 0, 'total': 0}
        
    branch_pc = 0xABC # Fixed PC

    history_buffer = deque([0] * history_window_size, maxlen=history_window_size) # For ML models

    for i in range(len(full_history)):
        actual_outcome = full_history[i]
        can_predict_ml_this_step = (i >= history_window_size)

        # --- Predict with Fusion Predictor FIRST ---
        
        
        if fusion_predictor:
            fusion_pred = fusion_predictor.predict(branch_pc, history_buffer, history_window_size)
            
            # *** FIXED LOGIC ***
            # Always increment total for fusion once a prediction is made
            results[fusion_label]['total'] += 1
            
            if fusion_pred == actual_outcome:
                results[fusion_label]['correct'] += 1
            # No need for complex logic based on who was chosen here,
            # just evaluate the final fusion prediction.


        # --- Predict with Standalone Basic Predictors ---
        basic_preds_this_step = {}
        for name, predictor in basic_predictors.items():
            basic_preds_this_step[name] = predictor.predict(branch_pc) # Store prediction
            if basic_preds_this_step[name] == actual_outcome:
                results[name]['correct'] += 1
            results[name]['total'] += 1


        # --- Predict with Standalone ML Models ---
        if can_predict_ml_this_step:
#             current_window = np.array(history_buffer).reshape(1, history_window_size, 1)
#             for name, model in ml_models.items():
#                 prob_taken = model.predict(current_window, verbose=0)[0][0]
#                 ml_pred = 1 if prob_taken >= 0.5 else 0
#                 if ml_pred == actual_outcome: results[name]['correct'] += 1
#                 results[name]['total'] += 1 # Total increments only after warm-up


        # --- UPDATE Phase ---
        # Update Fusion Predictor (which internally updates its P1 basic predictor)
        if fusion_predictor:
            # Pass the current history buffer state for correctness check inside update
            fusion_predictor.update(branch_pc, actual_outcome, history_buffer, history_window_size)

        # Update Standalone Basic Predictors
        for name, predictor in basic_predictors.items():
            predictor.update(branch_pc, actual_outcome)


        # --- Update History Buffer for NEXT iteration's ML predictions ---
        history_buffer.append(actual_outcome)

    # Calculate final accuracies
    accuracies = {}
    for name, res in results.items():
        if res['total'] > 0: accuracies[name] = res['correct'] / res['total']
        else: accuracies[name] = 0.0

    return accuracies

# --- Main Execution ---

if __name__ == "__main__":
    HISTORY_WINDOW_SIZE = 16
    DATA_FILE = 'branchnet_training_data.npz'

    # 1. Load Data
    if not os.path.exists(DATA_FILE):
        print(f"Data file {DATA_FILE} not found. Run generate_data.py (with noise=0).")
        exit()
    print(f"Loading data from {DATA_FILE}...")
    data = np.load(DATA_FILE)
    X_train_full = data['X']
    y_train_full = data['y']
    full_history = data['full_history']
    print(f"Loaded full history length: {len(full_history)}")

    # Split training data for ML models
    split_idx = int(len(X_train_full) * 0.8)
    X_val, y_val = X_train_full[split_idx:], y_train_full[split_idx:]
    X_train, y_train = X_train_full[:split_idx], y_train_full[:split_idx]

    # 2. Build and Train ML Models
    ml_models = {}
    epochs_to_train = 15 # Adjust as needed

    print("\n--- Building and Training ML Models ---")
    # --- CNN ---
    print("\nTraining Simple_BranchNet_CNN...")
    ml_models['BranchNet_CNN'] = build_simple_branchnet_cnn(HISTORY_WINDOW_SIZE)
    ml_models['BranchNet_CNN'].fit(X_train, y_train, epochs=epochs_to_train, batch_size=32, validation_data=(X_val, y_val), verbose=0)
    print("CNN Training Complete.")

    # --- CNN+LSTM ---
    print("\nTraining Simple_BranchNet_LSTM...")
    ml_models['BranchNet_LSTM'] = build_simple_branchnet_lstm(HISTORY_WINDOW_SIZE)
    ml_models['BranchNet_LSTM'].fit(X_train, y_train, epochs=epochs_to_train, batch_size=32, validation_data=(X_val, y_val), verbose=0)
    print("LSTM Training Complete.")

    # --- Transformer ---
    print("\nTraining Simple_BranchNet_Transformer...")
    ml_models['BranchNet_Transformer'] = build_simple_branchnet_transformer(HISTORY_WINDOW_SIZE)
    ml_models['BranchNet_Transformer'].fit(X_train, y_train, epochs=epochs_to_train, batch_size=32, validation_data=(X_val, y_val), verbose=0)
    print("Transformer Training Complete.")


    # 3. Initialize Basic Predictors (These will be used by Fusion)
    # We need fresh instances because the simulation modifies their state.
    basic_predictors_standalone = {
        "1-Bit": OneBitPredictor(),
        "2-Bit": TwoBitPredictor(),
        "2-Level": TwoLevelPredictor(ghr_bits=4),
        "Tournament": TournamentPredictor(ghr_bits=4)
    }

    # Initialize the specific instances for the ML-Aware Fusion predictor
    # ***** CHANGE THIS PART *****
    p1_for_fusion = TwoLevelPredictor(ghr_bits=4)     # Basic predictor (e.g., 2-Level)
    p2_for_fusion = ml_models['BranchNet_LSTM']       # The *trained Keras model* instance
    ml_model_label = 'BranchNet_LSTM'                 # Name for labeling
    # ***************************

    # Initialize the Fusion predictor (use the new class name)
    fusion_predictor_instance = FusionPredictorMLAware(
        p1_for_fusion,
        p2_for_fusion,
        ml_model_name=ml_model_label # Pass the name
    )
    print(f"\nFusion Predictor will combine: {fusion_predictor_instance.p1_name} and {fusion_predictor_instance.p2_name}")


    # 4. Run Simulation
    print("\nSimulating all predictors...")
    accuracies = simulate_predictors(
        full_history,
        basic_predictors_standalone, # Pass the dict for standalone comparison
        ml_models,                   # Pass the dict of trained ML models
        fusion_predictor=fusion_predictor_instance, # Pass the ML-aware fusion instance
        history_window_size=HISTORY_WINDOW_SIZE
    )

    # 5. Print Results
    print("\n--- Simulation Results ---")
    # ... (Print basic predictors) ...
    # ... (Print ML models) ...
    # Print basic predictors first
    for name in ["1-Bit", "2-Bit", "2-Level", "Tournament"]:
         print(f"{name:>20}: {accuracies.get(name, 0.0):.4f} Accuracy")
    # Print ML models
    for name in sorted(ml_models.keys()):
        print(f"{name:>20}: {accuracies.get(name, 0.0):.4f} Accuracy")
    # Generate the label used in results dict
    fusion_label = f"Fusion ({fusion_predictor_instance.p1_name}+{fusion_predictor_instance.p2_name})"
    if fusion_label in accuracies:
        print(f"{fusion_label:>20}: {accuracies[fusion_label]:.4f} Accuracy")

IndentationError: expected an indented block (2087232769.py, line 276)

In [11]:
for name in ["1-Bit", "2-Bit", "2-Level", "Tournament"]:
    print(f"{name:>20}: {accuracies.get(name, 0.0):.4f} Accuracy")
    # Print ML models
for name in sorted(ml_models.keys()):
    print(f"{name:>20}: {accuracies.get(name, 0.0):.4f} Accuracy")

               1-Bit: 0.4840 Accuracy
               2-Bit: 0.4290 Accuracy
             2-Level: 0.6480 Accuracy
          Tournament: 0.5900 Accuracy
       BranchNet_CNN: 0.7134 Accuracy
      BranchNet_LSTM: 1.0000 Accuracy
BranchNet_Transformer: 0.5437 Accuracy


In [4]:
import numpy as np

# Load the .npz file
data = np.load('branchnet_training_data.npz')

# Access the arrays within the .npz file
# .npz files are like dictionaries, so you access elements by their keys
print(data.files) # Prints the names of the arrays stored in the file

#Access a specific array
array1 = data['X'] # if the array was saved without a specific name
array2 = data['y'] # if the array was saved with the name 'name_of_your_array'
array3 = data['full_history']
# Print the arrays
print(array1)
print(array2)
print(array3)

# Close the file
data.close()

['X', 'y', 'full_history']
[[[0]
  [0]
  [0]
  ...
  [1]
  [0]
  [1]]

 [[0]
  [0]
  [1]
  ...
  [0]
  [1]
  [1]]

 [[0]
  [1]
  [1]
  ...
  [1]
  [1]
  [0]]

 ...

 [[0]
  [0]
  [0]
  ...
  [0]
  [0]
  [0]]

 [[0]
  [0]
  [1]
  ...
  [0]
  [0]
  [1]]

 [[0]
  [1]
  [1]
  ...
  [0]
  [1]
  [1]]]
[1 0 0 1 0 1 1 1 0 1 0 0 0 1 0 1 1 1 0 1 0 1 0 0 0 1 1 1 1 1 0 0 0 1 0 1 1
 1 0 1 1 0 0 1 0 0 1 0 1 1 1 0 1 0 1 0 0 0 1 0 1 0 1 0 1 0 0 0 0 0 1 0 1 1
 1 1 1 0 0 1 0 0 0 1 0 1 1 1 0 0 0 0 0 1 1 1 0 1 0 0 0 1 0 0 1 1 0 0 1 0 1
 0 1 0 0 0 1 1 1 1 1 0 0 1 1 0 1 1 0 0 1 1 0 1 1 0 0 1 1 0 1 1 0 0 1 1 0 1
 1 0 0 1 1 0 1 1 0 0 1 1 0 1 1 0 0 1 1 0 1 1 0 0 1 1 0 1 1 0 1 1 1 0 1 0 0
 0 1 0 1 1 1 0 1 0 1 0 1 0 1 0 1 1 1 0 1 1 0 0 0 0 0 1 1 1 1 0 0 0 0 0 1 0
 0 1 1 1 1 1 0 1 0 0 0 1 1 1 0 1 0 0 0 0 0 1 1 1 1 1 0 0 1 0 0 1 1 0 1 0 0
 0 1 0 1 1 0 1 0 0 0 1 0 0 1 1 0 0 1 0 1 1 1 0 1 0 0 1 1 0 1 0 1 0 0 0 1 0
 1 1 1 1 1 0 0 0 1 0 0 1 1 0 1 1 0 1 1 0 0 1 0 0 0 1 0 1 1 1 0 0 0 0 0 1 1
 1 0 1 0 0 0 1 0 0 1 1 0 1 1